In [1]:
# =========================
# PDAC: Companies enrichment + commodities/jurisdiction summariser
# - Input: PDAC_2026_Exhibitors_all_with_classification.csv (all entries, companies flagged)
# - Output (1): pdac_company_enriched.csv (companies only: Name, Exchange_Ticker, Website, Capacity, Classification)
# - Output (2): pdac_company_commodities.csv (companies only + top commodities/jurisdictions + evidence)
# - Output (3): pdac_summary_stats.json (aggregate counts)
# =========================

import re
import time
import json
import hashlib
import datetime as dt
from pathlib import Path
from urllib.parse import urljoin, urlparse

import pandas as pd
import requests
from bs4 import BeautifulSoup

# -------- PDF text extraction --------
PDF_OK = False
try:
    from pypdf import PdfReader
    PDF_OK = True
except Exception:
    PDF_OK = False

# -------------------------
# USER SETTINGS
# -------------------------
INPUT_CSV = r"C:\Users\julian.diaz\OneDrive - XENITH CONSULTING PTY LTD\Documents\01_BD\95_2026_PDAC\PDAC_2026_Exhibitors_all_with_classification.csv"

OUT_COMPANY_ENRICHED = r"C:\path\to\pdac_company_enriched.csv"
OUT_COMPANY_COMMS    = r"C:\path\to\pdac_company_commodities.csv"
OUT_SUMMARY_JSON     = r"C:\path\to\pdac_summary_stats.json"

DOWNLOAD_DIR = Path(r"C:\path\to\pdac_downloaded_pdfs")
DEBUG_DIR    = Path(r"C:\path\to\pdac_debug_pages")

ONLY_TICKERS = None  # e.g. ["BHP.AX"] for testing; else None

REQUEST_TIMEOUT = 25
SLEEP_S = 0.6
MAX_PAGES_TO_VISIT = 25
MAX_SITEMAP_URLS = 4000
MAX_SITEMAP_CHILDREN = 25
MAX_PDFS_PER_COMPANY = 6

VERBOSE = True
USER_AGENT = (
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
)

# -------------------------
# COMMODITY + JURISDICTION KEYWORDS
# -------------------------
COMMODITY_KW = {
    "Gold":      [r"\bgold\b", r"\bau\b"],
    "Copper":    [r"\bcopper\b", r"\bcu\b"],
    "Nickel":    [r"\bnickel\b", r"\bni\b"],
    "Lithium":   [r"\blithium\b", r"\bli\b", r"\bspodumene\b"],
    "Uranium":   [r"\buranium\b", r"\bu3o8\b"],
    "Silver":    [r"\bsilver\b", r"\bag\b"],
    "Zinc":      [r"\bzinc\b", r"\bzn\b"],
    "Lead":      [r"\blead\b", r"\bpb\b"],
    "REE":       [r"\brare earth", r"\bree\b", r"\bndpr\b", r"\bneodymium\b", r"\bpraseodymium\b"],
    "PGEs":      [r"\bplatinum\b", r"\bpalladium\b", r"\bpge\b"],
    "Graphite":  [r"\bgraphite\b"],
    "Tin":       [r"\btin\b", r"\bsn\b"],
    "Cobalt":    [r"\bcobalt\b", r"\bco\b"],
    "Manganese": [r"\bmanganese\b", r"\bmn\b"],
    "Iron Ore":  [r"\biron ore\b", r"\bhematite\b", r"\bmagnetite\b"],
    "Potash":    [r"\bpotash\b"],
}

# Keep it simple and useful: country/state mentions
JURISDICTION_KW = {
    "Canada":   [r"\bcanada\b", r"\bontario\b", r"\bquebec\b", r"\bbritish columbia\b", r"\balberta\b", r"\bnunavut\b", r"\byukon\b"],
    "USA":      [r"\busa\b", r"\bunited states\b", r"\bnevada\b", r"\balaska\b", r"\barizona\b"],
    "Australia":[r"\baustralia\b", r"\bwestern australia\b|\bwa\b", r"\bqueensland\b|\bqld\b", r"\bnsw\b|\bnew south wales\b", r"\bsa\b|\bsouth australia\b", r"\bnt\b|\bnorthern territory\b", r"\btas\b|\btasmania\b", r"\bvic\b|\bvictoria\b"],
    "Chile":    [r"\bchile\b"],
    "Peru":     [r"\bperu\b"],
    "Brazil":   [r"\bbrazil\b"],
    "Argentina":[r"\bargentina\b"],
    "Mexico":   [r"\bmexico\b"],
    "Africa":   [r"\bafrica\b", r"\bnamibia\b", r"\bbotswana\b", r"\bghana\b", r"\bsouth africa\b", r"\bdr congo\b|\bdrc\b", r"\bzambia\b"],
}

PDF_EXT_RE = re.compile(r"\.pdf(\?|$)", re.IGNORECASE)

REPORT_URL_HINT = re.compile(
    r"(investor|investors|asx|tsx|nyse|announce|announcement|release|news|media|report|results|financial|presentation|quarter|appendix|4c|4e|sedar)",
    re.IGNORECASE
)

# -------------------------
# HELPERS
# -------------------------
def log(msg: str):
    if VERBOSE:
        print(msg)

def normalize_site(site: str) -> str:
    site = (site or "").strip()
    if not site:
        return ""
    if not site.startswith(("http://", "https://")):
        site = "https://" + site
    return site.rstrip("/")

def short_hash(s: str) -> str:
    return hashlib.md5(s.encode("utf-8", errors="ignore")).hexdigest()[:10]

def safe_filename(s: str) -> str:
    s = re.sub(r"[^a-zA-Z0-9._-]+", "_", s)
    return s[:120]

def session() -> requests.Session:
    s = requests.Session()
    s.headers.update({"User-Agent": USER_AGENT, "Accept-Language": "en-US,en;q=0.9"})
    return s

def fetch_text(sess: requests.Session, url: str):
    try:
        r = sess.get(url, timeout=REQUEST_TIMEOUT, allow_redirects=True)
        ctype = (r.headers.get("content-type") or "").lower()
        return r.status_code, ctype, (r.text or "")
    except Exception:
        return 0, "", ""

def looks_like_pdf_bytes(first_bytes: bytes) -> bool:
    return first_bytes.startswith(b"%PDF-")

def download_pdf(sess: requests.Session, url: str, outpath: Path):
    try:
        r = sess.get(url, timeout=REQUEST_TIMEOUT, stream=True, allow_redirects=True)
        if r.status_code >= 400:
            return False, f"http_{r.status_code}"

        ctype = (r.headers.get("content-type") or "").lower()

        it = r.iter_content(chunk_size=1024 * 64)
        first = next(it, b"") or b""

        is_pdf = ("pdf" in ctype) or PDF_EXT_RE.search(url) or looks_like_pdf_bytes(first)
        if not is_pdf:
            return False, f"not_pdf_ctype={ctype[:40]}"

        outpath.parent.mkdir(parents=True, exist_ok=True)
        with open(outpath, "wb") as f:
            f.write(first)
            for chunk in it:
                if chunk:
                    f.write(chunk)

        if outpath.stat().st_size < 10_000:
            return False, "download_too_small"
        return True, "ok"
    except Exception as e:
        return False, f"exception_{type(e).__name__}"

def extract_pdf_text(pdf_path: Path) -> str:
    if not PDF_OK:
        return ""
    try:
        reader = PdfReader(str(pdf_path))
        parts = []
        for p in reader.pages[:40]:  # cap to reduce time
            t = p.extract_text() or ""
            if t:
                parts.append(t)
        return "\n".join(parts)
    except Exception:
        return ""

def extract_pdf_links_from_html(base_url: str, html: str):
    soup = BeautifulSoup(html, "lxml")
    out = []
    for a in soup.find_all("a", href=True):
        href = (a.get("href") or "").strip()
        if not href:
            continue
        absu = urljoin(base_url, href)
        label = " ".join(a.get_text(" ", strip=True).split())
        if PDF_EXT_RE.search(absu) or ("pdf" in absu.lower() and ("download" in absu.lower() or "uploads" in absu.lower() or "media" in absu.lower())):
            out.append({"url": absu, "label": label})
    return out

def get_sitemap_urls(sess: requests.Session, site: str):
    starts = ["/sitemap.xml", "/sitemap_index.xml", "/wp-sitemap.xml"]
    urls = []
    for p in starts:
        sm = site + p
        st, ct, txt = fetch_text(sess, sm)
        time.sleep(SLEEP_S)
        if st >= 400 or not txt or "xml" not in ct:
            continue
        soup = BeautifulSoup(txt, "xml")
        # sitemap index?
        smaps = soup.find_all("sitemap")
        if smaps:
            for smap in smaps[:MAX_SITEMAP_CHILDREN]:
                loc = smap.find("loc")
                if not loc:
                    continue
                child = loc.get_text(strip=True)
                st2, ct2, t2 = fetch_text(sess, child)
                time.sleep(SLEEP_S)
                if st2 >= 400 or not t2 or "xml" not in ct2:
                    continue
                s2 = BeautifulSoup(t2, "xml")
                for loc2 in s2.find_all("loc"):
                    u = loc2.get_text(strip=True)
                    if REPORT_URL_HINT.search(u):
                        urls.append(u)
                    if len(urls) >= MAX_SITEMAP_URLS:
                        return urls[:MAX_SITEMAP_URLS]
        else:
            for loc in soup.find_all("loc"):
                u = loc.get_text(strip=True)
                if REPORT_URL_HINT.search(u):
                    urls.append(u)
            if urls:
                return urls[:MAX_SITEMAP_URLS]
    return urls[:MAX_SITEMAP_URLS]

def discover_report_pages(site: str):
    paths = [
        "/investors", "/investor", "/investor-centre",
        "/asx-announcements", "/announcements", "/news", "/media",
        "/reports", "/financial-reports", "/presentations", "/results"
    ]
    return [site + p for p in paths]

def pick_pdf_candidates(sess: requests.Session, site: str, debug_dir: Path):
    debug_dir.mkdir(parents=True, exist_ok=True)
    notes = []
    pdfs = []

    seed_pages = get_sitemap_urls(sess, site)
    if seed_pages:
        notes.append(f"sitemap_filtered_pages={len(seed_pages)}")
    else:
        notes.append("no_sitemap_filtered_pages")
        seed_pages = discover_report_pages(site)

    visited = set()
    queue = seed_pages[:80]

    while queue and len(visited) < MAX_PAGES_TO_VISIT:
        url = queue.pop(0)
        if url in visited:
            continue
        visited.add(url)

        st, ct, html = fetch_text(sess, url)
        time.sleep(SLEEP_S)
        if st >= 400 or not html:
            continue

        dbg = debug_dir / f"{len(visited):02d}_{short_hash(url)}_{safe_filename(urlparse(url).path)}.html"
        dbg.write_text(html, encoding="utf-8", errors="ignore")

        pdfs.extend(extract_pdf_links_from_html(url, html))

        # expand a bit
        soup = BeautifulSoup(html, "lxml")
        added = 0
        for a in soup.find_all("a", href=True):
            href = (a.get("href") or "").strip()
            if not href:
                continue
            absu = urljoin(url, href)
            if absu in visited:
                continue
            if REPORT_URL_HINT.search(absu):
                queue.append(absu)
                added += 1
                if added >= 25:
                    break

    # dedupe
    seen = set()
    out = []
    for p in pdfs:
        u = p.get("url")
        if not u or u in seen:
            continue
        seen.add(u)
        out.append(p)

    notes += [f"visited_pages={len(visited)}", f"pdf_candidates={len(out)}"]
    # prioritize likely investor docs
    out.sort(key=lambda x: 0 if REPORT_URL_HINT.search((x.get("label","")+x.get("url",""))) else 1)
    return out[:MAX_PDFS_PER_COMPANY], notes

def score_keywords(text: str, kw_dict: dict):
    scores = {}
    evidence = {}
    for key, patterns in kw_dict.items():
        hits = 0
        ev = None
        for pat in patterns:
            m = re.search(pat, text, flags=re.I)
            if m:
                hits += len(re.findall(pat, text, flags=re.I))
                if ev is None:
                    # take short window around first hit
                    start = max(m.start() - 80, 0)
                    end   = min(m.end() + 120, len(text))
                    ev = re.sub(r"\s+", " ", text[start:end]).strip()
        if hits > 0:
            scores[key] = hits
            evidence[key] = ev or ""
    # top 3
    top = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:3]
    top_keys = [k for k, v in top]
    return top_keys, evidence

# --- Simple enrichment helpers (same style as earlier) ---
def clearbit_domain(company_name: str) -> str:
    url = "https://autocomplete.clearbit.com/v1/companies/suggest"
    try:
        r = requests.get(url, params={"query": company_name}, timeout=15)
        r.raise_for_status()
        data = r.json()
        if not data:
            return ""
        d = data[0].get("domain","") or ""
        if d and not d.startswith(("http://", "https://")):
            return "https://" + d
        return d
    except Exception:
        return ""

def yahoo_best_symbol(company_name: str) -> str:
    url = "https://query2.finance.yahoo.com/v1/finance/search"
    try:
        r = requests.get(url, params={"q": company_name, "quotesCount": 6, "newsCount": 0}, timeout=15)
        r.raise_for_status()
        data = r.json()
        quotes = data.get("quotes", []) or []
        for q in quotes:
            if q.get("quoteType") == "EQUITY" and q.get("symbol"):
                return q["symbol"]
        if quotes and quotes[0].get("symbol"):
            return quotes[0]["symbol"]
        return ""
    except Exception:
        return ""

def run():
    DOWNLOAD_DIR.mkdir(parents=True, exist_ok=True)
    DEBUG_DIR.mkdir(parents=True, exist_ok=True)

    df = pd.read_csv(INPUT_CSV)
    df.columns = [c.strip() for c in df.columns]

    # keep all entries for reference, but operate on companies only
    companies = df[df["Is_Company"].astype(str).str.upper().eq("YES")].copy()

    # optionally filter by ticker list
    if ONLY_TICKERS:
        companies = companies[companies["Exchange_Ticker"].fillna("").astype(str).isin(ONLY_TICKERS)].copy()

    # --- Enrich missing Website / Exchange_Ticker ---
    companies["Website"] = companies["Website"].fillna("").astype(str)
    companies["Exchange_Ticker"] = companies["Exchange_Ticker"].fillna("").astype(str)

    for i, row in companies.iterrows():
        name = str(row["Name"]).strip()

        if not row["Website"].strip():
            w = clearbit_domain(name)
            if w:
                companies.at[i, "Website"] = w

        if not row["Exchange_Ticker"].strip():
            t = yahoo_best_symbol(name)
            if t:
                companies.at[i, "Exchange_Ticker"] = t

        if VERBOSE and (i % 25 == 0):
            log(f"enriched {i}/{len(companies)}")
        time.sleep(0.2)

    companies.to_csv(OUT_COMPANY_ENRICHED, index=False, encoding="utf-8")
    log(f"Wrote: {OUT_COMPANY_ENRICHED}")

    # --- Commodity/Jurisdiction inference from PDFs ---
    sess = session()
    out_rows = []
    commodity_totals = {}
    jurisdiction_totals = {}

    for idx, r in companies.iterrows():
        name = str(r["Name"]).strip()
        tkr  = str(r["Exchange_Ticker"]).strip()
        site = normalize_site(str(r["Website"]).strip())
        if not site:
            out_rows.append({
                "Name": name, "Exchange_Ticker": tkr, "Website": "",
                "Top_Commodities": "", "Top_Jurisdictions": "",
                "Evidence": "", "Notes": "no_website"
            })
            continue

        log(f"\n[{len(out_rows)+1}/{len(companies)}] {tkr} - {name} ({site})")

        dbg_dir = DEBUG_DIR / (tkr or short_hash(name))
        pdf_list, notes = pick_pdf_candidates(sess, site, dbg_dir)

        combined_text = ""
        downloaded = 0
        for p in pdf_list:
            url = p.get("url","")
            if not url:
                continue
            outp = DOWNLOAD_DIR / (tkr or short_hash(name)) / f"{safe_filename(urlparse(url).path)}_{short_hash(url)}.pdf"
            ok, why = download_pdf(sess, url, outp)
            time.sleep(SLEEP_S)
            if ok:
                downloaded += 1
                combined_text += "\n" + extract_pdf_text(outp)
            else:
                notes.append(f"pdf_fail({why})")

        if not combined_text.strip():
            out_rows.append({
                "Name": name, "Exchange_Ticker": tkr, "Website": site,
                "Top_Commodities": "", "Top_Jurisdictions": "",
                "Evidence": "", "Notes": "; ".join(notes + ["no_pdf_text"])
            })
            continue

        top_comms, comm_ev = score_keywords(combined_text, COMMODITY_KW)
        top_jurs, jur_ev   = score_keywords(combined_text, JURISDICTION_KW)

        for c in top_comms:
            commodity_totals[c] = commodity_totals.get(c, 0) + 1
        for j in top_jurs:
            jurisdiction_totals[j] = jurisdiction_totals.get(j, 0) + 1

        # build short evidence string
        ev_parts = []
        for c in top_comms[:2]:
            if comm_ev.get(c):
                ev_parts.append(f"[{c}] {comm_ev[c][:220]}")
        for j in top_jurs[:1]:
            if jur_ev.get(j):
                ev_parts.append(f"[{j}] {jur_ev[j][:220]}")
        evidence = " | ".join(ev_parts)

        out_rows.append({
            "Name": name,
            "Exchange_Ticker": tkr,
            "Website": site,
            "Capacity": r.get("Capacity",""),
            "Classification": r.get("Classification",""),
            "Top_Commodities": ", ".join(top_comms),
            "Top_Jurisdictions": ", ".join(top_jurs),
            "Evidence": evidence,
            "Notes": "; ".join(notes + [f"pdf_downloaded={downloaded}"])
        })

    out_df = pd.DataFrame(out_rows)
    out_df.to_csv(OUT_COMPANY_COMMS, index=False, encoding="utf-8")
    log(f"\nWrote: {OUT_COMPANY_COMMS}")

    summary = {
        "companies_total": int(len(companies)),
        "companies_with_website": int((companies["Website"].astype(str).str.len() > 0).sum()),
        "companies_with_ticker": int((companies["Exchange_Ticker"].astype(str).str.len() > 0).sum()),
        "commodity_totals_top": dict(sorted(commodity_totals.items(), key=lambda x: x[1], reverse=True)[:15]),
        "jurisdiction_totals_top": dict(sorted(jurisdiction_totals.items(), key=lambda x: x[1], reverse=True)[:15]),
    }
    Path(OUT_SUMMARY_JSON).write_text(json.dumps(summary, indent=2), encoding="utf-8")
    log(f"Wrote: {OUT_SUMMARY_JSON}")

if __name__ == "__main__":
    run()

enriched 0/552
enriched 25/552
enriched 50/552
enriched 75/552
enriched 100/552
enriched 125/552
enriched 150/552
enriched 225/552
enriched 250/552
enriched 275/552
enriched 300/552
enriched 325/552
enriched 350/552
enriched 375/552
enriched 400/552
enriched 425/552
enriched 450/552
enriched 525/552
enriched 575/552
enriched 600/552
enriched 625/552
enriched 650/552
enriched 725/552
enriched 850/552
enriched 1025/552
Wrote: C:\path\to\pdac_company_enriched.csv

[1/552]  - 1911 Gold Corporation (https://gcl.lk)

[6/552]  - Corporation (https://corporationwiki.com)

[9/552]  - Agnico Eagle Mines Limited (https://agnicoeagle.com)

[10/552]  - AIL Mining (https://ailmining.com)

[22/552]  - American Lithium Corp. (https://americanlithiumcorp.com)

[30/552]  - Uranium Corp. (https://ucil.gov.in)

[32/552]  - Arizona Metals Corp. (https://arizonametalscorp.com)

[34/552]  - Artemis Gold Inc. (https://artemisgoldinc.com)

[47/552]  - AURYN Mining Corp. (https://aurynminingcorp.com)

[48/552] 

Ignoring wrong pointing object 24 0 (offset 0)
Ignoring wrong pointing object 26 0 (offset 0)
Ignoring wrong pointing object 28 0 (offset 0)
Ignoring wrong pointing object 30 0 (offset 0)
Ignoring wrong pointing object 32 0 (offset 0)
Ignoring wrong pointing object 282 0 (offset 0)
Ignoring wrong pointing object 284 0 (offset 0)
Ignoring wrong pointing object 357 0 (offset 0)



[187/552]  - Fury Gold Mines (https://furygoldmines.com)


Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 24 0 (offset 0)
Ignoring wrong pointing object 27 0 (offset 0)
Ignoring wrong pointing object 37 0 (offset 0)
Ignoring wrong pointing object 44 0 (offset 0)
Ignoring wrong pointing object 73 0 (offset 0)



[189/552]  - Communications Inc. (https://communicationsinc.co.uk)

[190/552]  - Galiano Gold (https://galianogold.com)

[192/552]  - Galway Metals Inc. (https://galwaymetalsinc.com)

[199/552]  - LLC (https://garant.ru)

[202/552]  - Minerals) (https://minerals.net)

[205/552]  - Geotech Ltd. (https://geotechltd.com)

[219/552]  - Goliath Resources Limited (https://goliathresourcesltd.com)

[221/552]  - Partners Inc. (https://partnersbend.org)

[225/552]  - Grid Metals Corp. (https://gridmetalscorp.com)

[236/552]  - Honey Badger Silver (https://honeybadgersilver.com)

[239/552]  - HydroTech Mining (https://hydrotechmining.com)

[246/552]  - Inventus Mining (https://inventusmining.com)

[252/552]  - Ivanhoe Mines (https://ivanhoemines.com)

[253/552]  - Jaguar Mining (https://jaguarmining.com)

[256/552]  - Jindalee Lithium (https://jindaleelithium.com)

[258/552]  - Juggernaut Exploration Ltd. (https://juggernautexploration.com)

[273/552]  - KPI Mining Solutions (https://kpimining.

...


[277/552]  - Lahontan Gold Corp. (https://lahontangoldcorp.com)


Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 15 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 21 0 (offset 0)
Ignoring wrong pointing object 24 0 (offset 0)
Ignoring wrong pointing object 26 0 (offset 0)
Ignoring wrong pointing object 33 0 (offset 0)
Ignoring wrong pointing object 35 0 (offset 0)
Ignoring wrong pointing object 37 0 (offset 0)
Ignoring wrong pointing object 39 0 (offset 0)
Ignoring wrong pointing object 41 0 (offset 0)
Ignoring wrong pointing object 49 0 (offset 0)
Ignoring wrong pointing object 51 0 (offset 0)
Ignoring wrong pointing object 56 0 (offset 0)
Ignoring wrong pointing object 61 0 (offset 0)
Ignoring wrong pointing object 63 0 (offset 0)
Ignoring wrong pointing object 65 0 (offset 0)
Ignoring wrong 


[293/552]  - Mako Mining Corp. (https://makominingcorp.com)

[296/552]  - Manganese X Energy Corp. (https://manganesexenergycorp.com)

[298/552]  - Maritime Resources Corp. (https://maritimeresourcescorp.com)

[306/552]  - McFarlane Lake Mining (https://mcfarlanelakemining.com)

[310/552]  - Services Limited (https://servicesltd.co.uk)

[323/552]  - Mining Finland (https://miningfinland.com)

[326/552]  - Natural Resources (https://nrar.nsw.gov.au)

[344/552]  - New Age Metals (https://newagemetals.com)

[352/552]  - Nickel North Exploration (https://nnexploration.com)

[359/552]  - Northstar Gold Corp. (https://northstargoldcorp.com)

[361/552]  - Novamera Inc. (https://novamerainc.com)

[363/552]  - Nuinsco Resources Limited (https://nuinsco.ca)

[370/552]  - Onyx Gold (https://onyxgoldsmiths.co.uk)

[375/552]  - Orosur Mining (https://orosur.ca)

[379/552]  - Management Limited (https://720ltd.com)

[399/552]  - Prospector Metals Corp. (https://prospectormetalscorp.com)

[404/552] 

Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 26 0 (offset 0)
Ignoring wrong pointing object 28 0 (offset 0)
Ignoring wrong pointing object 39 0 (offset 0)
Ignoring wrong pointing object 41 0 (offset 0)
Ignoring wrong pointing object 43 0 (offset 0)
Ignoring wrong pointing object 53 0 (offset 0)
Ignoring wrong pointing object 84 0 (offset 0)
Ignoring wrong pointing object 86 0 (offset 0)
Ignoring wrong pointing object 88 0 (offset 0)
Ignoring wrong pointing object 92 0 (offset 0)
Ignoring wrong pointing object 112 0 (offset 0)
Ignoring wrong pointing object 127 0 (offset 0)
Ignoring wrong pointing object 129 0 (offset 0)
Ignoring wrong pointing object 131 0 (offset 0)
Ignoring wrong pointing object 133 0 (offset 0)
Ignoring wrong pointing object 135 0 (offset 0)
Ignoring wrong pointing object 137 0 (offset 0)
Ignoring wrong pointing object 140 0 (offset 0)
Ignoring wrong pointing object 142 0 (offset 0)
Ignori


[475/552]  - Strikepoint Gold (https://strikepointgold.com)

[476/552]  - Sun Peak Metals (https://sunpeakmetals.com)

[478/552]  - Surge Battery Metals (https://surgebatterymetals.com)

[480/552]  - T2 Metals (https://t2metals.com)

[489/552]  - Tek84 Inc. (https://tekincogroup.com)

[497/552]  - The Metals Company (https://metals.co)

[499/552]  - Thunder Gold Corp. (https://thundergoldcorp.com)

[501/552]  - Transition Metals Corp. (https://transitionmetalscorp.com)

[504/552]  - Triumph Gold Corp. (https://triumphgoldcorp.com)

[507/552]  - Trade Inc. (https://tradein.com)

[510/552]  - LLC (https://garant.ru)

[518/552]  - Zijin Mining Group (https://zjky.cn)

[523/552]  - Canada Inc. (https://canadaincredible.com)

[533/552]  - Canada Inc. (https://canadaincredible.com)

[539/552]  - Viva Gold Corp. (https://vivagoldcorp.com)

[540/552]  - Vizsla Silver Corp. (https://vizslasilvercorp.com)

[546/552]  - Western Exploration (https://westernexploration.com)

[547/552]  - Vanadium 